# Title: Personalized Medical Recommendation System with Machine Learning

# Description:

Welcome to our cutting-edge Personalized Medical Recommendation System, a powerful platform designed to assist users in understanding and managing their health. Leveraging the capabilities of machine learning, our system analyzes user-input symptoms to predict potential diseases accurately.

# load dataset & tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle

In [5]:
data = pd.read_csv('Training.csv')

In [4]:
data

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [21]:
# vals = dataset.values.flatten()

In [6]:
X = data.drop('prognosis', axis=1)
y = data['prognosis']

# train test split

In [7]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=20)

# Training top models

In [9]:
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# single prediction

In [10]:
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

In [11]:
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.4f}")

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Accuracy: 1.0000


In [12]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 1.0000


In [13]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(np.array2string(cm, separator=', '))

Confusion Matrix:
[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]


In [14]:
with open('nithil_rf.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Recommendation System and Prediction

# load database and use logic for recommendations

In [15]:
with open('nithil_rf.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [16]:
print("Test 1:")
pred1 = le.inverse_transform(loaded_model.predict(X_test.iloc[0].values.reshape(1, -1)))[0]
print(f"Predicted Disease: {pred1}")
print(f"Actual Disease: {le.inverse_transform([y_test[0]])[0]}")

Test 1:
Predicted Disease: hepatitis A
Actual Disease: hepatitis A


c:\Users\nithi\anaconda3\envs\nithil\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [17]:
print("Test 2:")
pred2 = le.inverse_transform(loaded_model.predict(X_test.iloc[100].values.reshape(1, -1)))[0]
print(f"Predicted Disease: {pred2}")
print(f"Actual Disease: {le.inverse_transform([y_test[100]])[0]}")

Test 2:
Predicted Disease: Varicose veins
Actual Disease: Varicose veins


c:\Users\nithi\anaconda3\envs\nithil\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [18]:
sym_des = pd.read_csv("C:\\Users\\nithi\\OneDrive\\Desktop\\Final\\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\\symtoms_df.csv")
precautions = pd.read_csv("C:\\Users\\nithi\\OneDrive\\Desktop\\Final\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\\precautions_df.csv")
workout = pd.read_csv("C:\\Users\\nithi\\OneDrive\\Desktop\\Final\\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\\workout_df.csv")
description = pd.read_csv("C:\\Users\\nithi\\OneDrive\\Desktop\\Final\\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\\description.csv")
medications = pd.read_csv("C:\\Users\\nithi\\OneDrive\\Desktop\\Final\\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\\medications.csv")
diets = pd.read_csv("C:\\Users\\nithi\\OneDrive\\Desktop\\Final\\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\\diets.csv")

In [19]:
def helper(dis):
    desc = description[description['Disease'] == dis]['Description'].iloc[0] if not description[description['Disease'] == dis].empty else "No description available"
    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].values.flatten() if not precautions[precautions['Disease'] == dis].empty else []
    med = medications[medications['Disease'] == dis]['Medication'].values if not medications[medications['Disease'] == dis].empty else []
    die = diets[diets['Disease'] == dis]['Diet'].values if not diets[diets['Disease'] == dis].empty else []
    wrkout = workout[workout['disease'] == dis]['workout'].values if not workout[workout['disease'] == dis].empty else []
    return desc, pre, med, die, wrkout

In [20]:
symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}

In [21]:
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    for item in patient_symptoms:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
    return le.inverse_transform(loaded_model.predict([input_vector]))[0]

In [22]:
for _ in range(2):  # Run twice as in your original code
    symptoms = input("Enter your symptoms (comma-separated): ")
    user_symptoms = [s.strip() for s in symptoms.split(',')]
    user_symptoms = [symptom.strip("[]' ") for symptom in user_symptoms]
    predicted_disease = get_predicted_value(user_symptoms)

    desc, pre, med, die, wrkout = helper(predicted_disease)

    print("=================Predicted Disease============")
    print(predicted_disease)
    print("=================Description==================")
    print(desc)
    print("=================Precautions==================")
    for i, p in enumerate(pre, 1):
        print(f"{i}: {p}")
    print("=================Medications==================")
    for i, m in enumerate(med, 1):
        print(f"{i}: {m}")
    print("=================Workout==================")
    for i, w in enumerate(wrkout, 1):
        print(f"{i}: {w}")
    print("=================Diets==================")
    for i, d in enumerate(die, 1):
        print(f"{i}: {d}")
    print("\n" + "="*40 + "\n")

c:\Users\nithi\anaconda3\envs\nithil\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


=================Predicted Disease============
AIDS
=================Description==================
AIDS (Acquired Immunodeficiency Syndrome) is a disease caused by HIV that weakens the immune system.
=================Precautions==================
1: avoid open cuts
2: wear ppe if possible
3: consult doctor
4: follow up
=================Medications==================
1: ['Antiretroviral drugs', 'Protease inhibitors', 'Integrase inhibitors', 'Entry inhibitors', 'Fusion inhibitors']
=================Workout==================
1: Follow a balanced and nutritious diet
2: Include lean proteins
3: Consume nutrient-rich foods
4: Stay hydrated
5: Include healthy fats
6: Avoid raw or undercooked foods
7: Limit sugary foods and beverages
8: Consume immune-boosting foods
9: Take prescribed supplements
10: Consult a healthcare professional
=================Diets==================
1: ['Balanced Diet', 'Protein-rich foods', 'Fruits and vegetables', 'Whole grains', 'Healthy fats']


=================Pre

c:\Users\nithi\anaconda3\envs\nithil\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
